# 1-D Mixed formulation for second order equations (Unit 2.5)
## Not possible - HDiv not supported for 1-D
## (requires NGSolve to be compiled with umfpack)
We consider the diffusion equation

\begin{align*}
−\text{div}\lambda \nabla u &= f \text{ in } \Omega\\
u &= u_D \text{ on } \Gamma_D \text{  (Dirichlet)}\\
\lambda \frac{\partial u}{\partial n} &= g \text{ on } \Gamma_N  \text{  (Neumann)}
\end{align*}


### Primal variational formulation

Find $u\in H^1$, $u=u_D$ on $\Gamma_D$

such that
$$ \int_\Omega \lambda \nabla u \nabla v = \int_\Omega f v + \int_{\Gamma_N} g v, \ \forall v \in H^1, v=0 \text{ on } \Gamma_D$$

### First order system

Find scalar $u$ and the flux $\sigma$ such that

$$\lambda^{-1} \sigma = \nabla u, \text{div } \sigma = -f$$

with boundary conditions
$$\sigma \cdot n = g \text{ on } \Gamma_N, \text{ and } u = u_D \text{ on } \Gamma_D$$

### Mixed variational formulation

Find $(\sigma, u) \in H(div) \times L_2$ such that $\sigma_n = g$ on $\Gamma_N$ and

$$\int_\Omega \lambda^{-1} \sigma \tau + \text{ div } \sigma v + \text{ div }\tau u = - \int_\Omega fv + \int_{\Gamma_D} u_D \tau_n$$


for all test-functions $(\tau, v) \in H(div) \times L_2$ with $\tau_n = 0$.

Here $\sigma_n$ is the normal trace operator $\sigma \cdot n |_{\Gamma_N}$, which is meaningful in $H(div)$.

### Notes:
Question: is $\lambda$ a Lagrange parameter? - as mentioned in the facet spaces and hybrid methods tutorial?  I don't quite get this...

From Arnold: For some problems, such as the Stokes problem, primal-based methods are impractical.  For such problems the mixed methods are the simplest and most direct alternative and are widely used.  For the other examples, however, the most basic methods are primal-based.  A commonly stated reason to prefer mixed methods in these cases is that the dual variable (stress or elasticity, flux for thermal problems, moments for plate bending) is often the variable
of most interest.  For primal-based methods this variable is not a fundamental unknown and must be obtained a posteriori by differentiation, which entails a loss of accuracy.  For the mixed methods, however, the dual variable is computed as a fundamental unknown. Of course, this argument is only heuristic.  Its correctness depends on the available mixed finite  element  spaces  and  primal  finite  element  spaces,  the  accuracy  they  offer,  and  the computational work they require to solve the corresponding discrete problems. Another  common  motivation  for  the  use  of  mixed  methods  is  the  avoidance  of $C^1$ elements for plate bending and other fourth order problems.  This is because the mixed functional for plate bending involves no more than two derivatives in any term and hence, after a suitable integration by parts, may be evaluated on finite element spaces with merely continuous elements.  The primal variational functional, however, requires the use of $C^1$ elements (or non-conforming elements).

From Advanced FE lecture notes, Eric Sonnendrucker, Ahmed Ratnani:
The $H(div)$ space consists of vector fields.  Each element has n components in n
dimensions.  Vector fields have a continuous normal component and a discontinuous tangential component. According to Jay, $$H(div) = \lbrace q \in L_2 : \text{div}\ q \in L_2 \rbrace$$  
If $u \in H(div)$ and $v \in L_2$, the variational form for a problem in $H(div)$ space is:
$$\int_{\Omega} (\text{div} u) v \equiv \int_{\Omega} (\nabla \cdot u) v = -\int_{\Omega} u \cdot \nabla v + \int_{\partial \Omega} (u \cdot n) v $$ 

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
%gui tk

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [10]:
source = sin(3.14*x)
ud = CoefficientFunction(5)
g = CoefficientFunction([y*(1-y) if bc=="left" else 0 for bc in mesh.GetBoundaries()])
lam = 10

We will demonstrate that the primal and mixed problems give approximately the same solutions for $u$ and flux $\nabla u$.


Setup and solve primal problem:

In [11]:
fesp = H1(mesh, order=4, dirichlet="bottom")
up = fesp.TrialFunction()
vp = fesp.TestFunction()

ap = BilinearForm(fesp)
ap += SymbolicBFI(lam * grad(up) * grad(vp))
ap.Assemble()

fp = LinearForm(fesp)
fp += SymbolicLFI(source * vp)
fp += SymbolicLFI(g * vp, BND)
fp.Assemble()

gfup = GridFunction(fesp, "u-primal")
gfup.Set(ud, BND)

# Solve the boundary value problem
#r = fp.vec.CreateVector()
#r.data = fp.vec - ap.mat * gfup.vec
#gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
fp.vec.data -= ap.mat * gfup.vec
gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * fp.vec
Draw (gfup)
Draw (lam * grad(gfup), mesh, "flux-primal")

### Notes: 
If $\lambda = 1$, to satisfy the boundary condition, the value of $\frac{\partial u}{\partial n} \equiv \nabla u \cdot n$ on the left boundary must equal $y(1-y)$.  This function is zero at $y=0$ and $y=1$ with max value $1/4$ at $y = 1/2$.  We can see that the solution is reasonable since at the bottom left, the magnitude of the gradient is large, but its direction is vertically upward.  At the top left, the direction of the gradient is outward, but its magnitude approaches zero.  On the top and right edges, we see also that the normal derivative is zero.

It's interesting that changing lambda doesn't affect the character of the solution at all, it only scales u.  The larger the value of $\lambda$, the 'flatter' the solution.  For example, if $\lambda = 10$, the min and max values of u are $5$ and $5.04$, if $\lambda = 0.1$, the min and max values of $u$ are $5.1$ and $9.2$ and if $\lambda = .001$, the min and max values of $u$ are $22$ and $430$.  The direction of the gradient is unchanged.

Define spaces for mixed problem. Note that essential boundary conditions are set to the H(div)-component on the opposite boundary. Creating a space from a list of spaces generates a product space:

In [12]:
V = HDiv(mesh, order=4, dirichlet="right|top|left")
Q = L2(mesh, order=3)
fesm = FESpace([V,Q])

The space provides now a list of trial and test-functions:

In [13]:
sigma, u = fesm.TrialFunction()
tau, v = fesm.TestFunction()

The normal vector is provided as a special coefficient function (which may be used only at the boundary). The orientation depends on the definition of the geometry. In 2D, it is the tangential vector rotated to the right, and is the outer vector in our case. Since every CoefficientFunction must know its vector-dimension, we have to provide the spatial dimension:

In [14]:
normal = specialcf.normal(mesh.dim)
print (normal)

coef 14NormalVectorCFILi2EE, real, dim=2



Define the forms on the product space. For the boundary term, we have to use the Trace operator, which provides the projection to normal direction.

In [7]:
am = BilinearForm(fesm)
am += SymbolicBFI(1/lam * sigma * tau + div(sigma)*v + div(tau)* u)
am.Assemble()
fm = LinearForm(fesm)
fm += SymbolicLFI(-source*v)
fm += SymbolicLFI(ud * (tau.Trace()*normal), BND)
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")

The proxy-functions used for the forms know which component of the product space they belong to. To access components of the solution, we have to unpack its components. They don't have their own coefficient vectors, but refer to ranges of the big coefficient vector.

In [8]:
gfsigma = gfm.components[0]
gfu = gfm.components[1]

Just to try something:

In [9]:
gfsigma.Set(CoefficientFunction( (sin(10*x), sin(10*y)) ))
gfu.Set (x)
Draw (gfsigma, mesh, "flux-mixed")
Draw (gfu, mesh, "u-mixed")


Now set the essential boundary conditions for the flux part:

In [10]:
gfsigma.Set(g*normal, BND)

In [11]:
fm.vec.data -= am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * fm.vec
Redraw()


Calculate the difference:

In [12]:
print ("err-u:   ", sqrt(Integrate( (gfup-gfu)*(gfup-gfu), mesh)))
errflux = lam * grad(gfup) - gfsigma
print ("err-flux:", sqrt(Integrate(errflux*errflux, mesh)))

err-u:    3.9686871319361595e-08
err-flux: 3.639053243656167e-05
